# ELEC 475 Lab 4 - CLIP Visualization Generator

This notebook generates all required visualizations for Lab 4 Section 2.4:
1. Text→Image retrieval (including 'sport' and 'a dog playing')
2. Zero-shot image classification

**Features:**
- Clones Lab4 code from GitHub
- Downloads ONLY val2014 images (~6GB, not entire 20+GB dataset)
- Handles different model architectures (base, batchnorm, dropout)
- Saves outputs to Google Drive

## Setup: Mount Google Drive and Install Dependencies

In [26]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
# Install required packages
!pip install -q transformers torch torchvision tqdm pillow matplotlib

## Clone Lab4 Code from GitHub

In [28]:
import os

# Clone repository
if not os.path.exists('475_ML-CV_Labs'):
    print("Cloning repository from GitHub...")
    !git clone https://github.com/Jcub05/475_ML-CV_Labs.git
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")

# Navigate to Lab4 directory
os.chdir('/content/475_ML-CV_Labs/Lab4')
print(f"✓ Current directory: {os.getcwd()}")

# Verify required files exist
required_files = ['model.py', 'model_modified.py', 'visualize.py', 'metrics.py']
for f in required_files:
    if os.path.exists(f):
        print(f"✓ {f} found")
    else:
        print(f"✗ {f} missing!")

✓ Repository already exists
✓ Current directory: /content/475_ML-CV_Labs/Lab4
✓ model.py found
✓ model_modified.py found
✓ visualize.py found
✓ metrics.py found


## Download ONLY val2014 Images (~6GB)

This downloads only the validation images, not the entire 20+GB dataset.

In [29]:
import urllib.request
import zipfile
from pathlib import Path

# Setup directories
data_dir = Path('/content/coco_data')
data_dir.mkdir(exist_ok=True)

val_zip_path = data_dir / 'val2014.zip'
val_dir = data_dir / 'val2014'

# Download val2014 images if not already downloaded
if not val_dir.exists() or len(list(val_dir.glob('*.jpg'))) == 0:
    print("Downloading COCO val2014 images (~6GB)...")
    print("This will take 5-10 minutes depending on connection speed.")

    # Download from official COCO website
    val_url = 'http://images.cocodataset.org/zips/val2014.zip'

    # Download with progress
    def download_progress(block_num, block_size, total_size):
        downloaded = block_num * block_size
        percent = min(100, downloaded * 100 / total_size)
        print(f'\rDownloading: {percent:.1f}% ({downloaded / 1e9:.2f} GB / {total_size / 1e9:.2f} GB)', end='')

    urllib.request.urlretrieve(val_url, val_zip_path, download_progress)
    print("\n✓ Download complete")

    # Extract
    print("\nExtracting val2014.zip...")
    with zipfile.ZipFile(val_zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)
    print("✓ Extraction complete")

    # Remove zip file to save space
    val_zip_path.unlink()
    print("✓ Removed zip file to save space")
else:
    print("✓ val2014 images already downloaded")

# Verify
num_val_images = len(list(val_dir.glob('*.jpg')))
print(f"\n✓ Validation images found: {num_val_images}")
print(f"✓ Image directory: {val_dir}")

✓ val2014 images already downloaded

✓ Validation images found: 40504
✓ Image directory: /content/coco_data/val2014


## Upload Your Trained Model Checkpoint & Configure Model Type

**IMPORTANT:** Set `MODEL_TYPE` to match your checkpoint:
- `'base'` - Standard model (no batchnorm, no dropout)
- `'batchnorm'` - Model with BatchNorm in projection
- `'dropout'` - Model with Dropout in projection
- `'batchnorm_dropout'` - Model with both BatchNorm and Dropout
- `'deeper'` - Model with 3-layer projection
- `'all_combined'` - All modifications combined

In [30]:
from google.colab import files

# # ============================================
# # CONFIGURATION: Set this to match your model!
# # ============================================
# MODEL_TYPE = 'batchnorm'  # Change to: 'base', 'batchnorm', 'dropout', 'batchnorm_dropout', 'deeper', 'all_combined'

# print(f"Model type set to: {MODEL_TYPE}")
# print("\nUpload your trained model checkpoint (e.g., best_model.pth)")
# uploaded = files.upload()
# model_checkpoint_path = list(uploaded.keys())[0]
# print(f"✓ Model checkpoint: {model_checkpoint_path}")

# Alternative: If checkpoint is in Google Drive, uncomment and modify:
MODEL_TYPE = 'batchnorm'
model_checkpoint_path = '/content/drive/MyDrive/elec475_lab4/models/best_model_batch_norm.pth'

## Visualization Code with Multi-Architecture Support

In [31]:
import torch
from transformers import CLIPProcessor, CLIPModel
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

from model import CLIPFineTuneModel
from model_modified import CLIPImageEncoderModified, CLIPFineTuneModelModified
from visualize import (
    visualize_text_to_image_retrieval,
    zero_shot_classification,
    create_retrieval_grid
)

# Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
output_dir = Path('/content/Visualizations')
output_dir.mkdir(exist_ok=True)
num_images = 1000  # Number of val images to use

print(f"Device: {device}")
print(f"Output directory: {output_dir}")
print(f"Model type: {MODEL_TYPE}")

Device: cuda
Output directory: /content/Visualizations
Model type: batchnorm


In [32]:
def get_clip_transform():
    """Get CLIP preprocessing transform."""
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.48145466, 0.4578275, 0.40821073),
            std=(0.26862954, 0.26130258, 0.27577711)
        )
    ])


def load_model_with_architecture(model_path, model_type, device):
    """
    Load model with correct architecture based on model_type.

    Args:
        model_path: Path to checkpoint
        model_type: One of 'base', 'batchnorm', 'dropout', etc.
        device: Device to load model on
    """
    print(f"\nLoading {model_type} model from {model_path}...")

    # Model architecture configurations
    MODEL_CONFIGS = {
        'base': {
            'use_batchnorm': False,
            'use_dropout': False,
            'deeper_projection': False,
            'learnable_temperature': False,
        },
        'batchnorm': {
            'use_batchnorm': True,
            'use_dropout': False,
            'deeper_projection': False,
            'learnable_temperature': False,
        },
        'dropout': {
            'use_batchnorm': False,
            'use_dropout': True,
            'dropout_rate': 0.1,
            'deeper_projection': False,
            'learnable_temperature': False,
        },
        'batchnorm_dropout': {
            'use_batchnorm': True,
            'use_dropout': True,
            'dropout_rate': 0.1,
            'deeper_projection': False,
            'learnable_temperature': False,
        },
        'deeper': {
            'use_batchnorm': False,
            'use_dropout': False,
            'deeper_projection': True,
            'learnable_temperature': False,
        },
        'all_combined': {
            'use_batchnorm': True,
            'use_dropout': True,
            'dropout_rate': 0.1,
            'deeper_projection': True,
            'learnable_temperature': True,
        }
    }

    if model_type == 'base':
        # Use standard CLIPFineTuneModel (no modifications)
        model = CLIPFineTuneModel(
            embed_dim=512,
            pretrained_resnet=True,
            clip_model_name="openai/clip-vit-base-patch32",
            freeze_text_encoder=True
        ).to(device)
    else:
        # Use modified architecture
        if model_type not in MODEL_CONFIGS:
            raise ValueError(f"Unknown model type: {model_type}. Choose from {list(MODEL_CONFIGS.keys())}")

        config = MODEL_CONFIGS[model_type]

        # Create modified image encoder
        image_encoder = CLIPImageEncoderModified(
            embed_dim=512,
            **config
        )

        # Load CLIP text encoder
        clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

        # Create modified model
        model = CLIPFineTuneModelModified(
            image_encoder=image_encoder,
            text_encoder=clip_model.text_model,
            tokenizer=None
        ).to(device)

    # Load checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    state_dict = checkpoint.get('model_state_dict', checkpoint)

    # Load with relaxed matching (in case of minor differences)
    missing_keys, unexpected_keys = model.load_state_dict(state_dict, strict=False)

    if missing_keys:
        print(f"⚠ Warning: Missing keys in checkpoint: {missing_keys[:3]}...")
    if unexpected_keys:
        print(f"⚠ Warning: Unexpected keys in checkpoint: {unexpected_keys[:3]}...")

    # CRITICAL: Set to eval mode (especially important for BatchNorm/Dropout!)
    model.eval()

    print(f"✓ {model_type} model loaded and set to eval mode")
    return model


def precompute_image_embeddings(model, image_paths, transform, device, batch_size=32):
    """Precompute embeddings for all images."""
    print(f"\nPrecomputing embeddings for {len(image_paths)} images...")

    all_embeds = []
    model.eval()

    with torch.no_grad():
        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i+batch_size]

            images = []
            for path in batch_paths:
                img = Image.open(path).convert('RGB')
                images.append(transform(img))

            images = torch.stack(images).to(device)

            # Use encode_image method (works for both base and modified models)
            embeds = model.encode_image(images).cpu()
            all_embeds.append(embeds)

            if (i // batch_size + 1) % 10 == 0:
                print(f"  Processed {i+len(batch_paths)}/{len(image_paths)}")

    all_embeds = torch.cat(all_embeds, dim=0)
    print(f"✓ Embeddings computed: {all_embeds.shape}")
    return all_embeds


# Wrapper class for modified models to work with visualize.py
class ModifiedModelWrapper:
    """Wrapper to make modified models compatible with visualization functions."""
    def __init__(self, model, processor):
        self.model = model
        self.processor = processor

    def eval(self):
        self.model.eval()
        return self

    def encode_text(self, input_ids, attention_mask):
        """Encode text using the model's text encoder."""
        with torch.no_grad():
            # For modified models, use the text_encoder directly
            if hasattr(self.model, 'text_encoder'):
                outputs = self.model.text_encoder(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )
                import torch.nn.functional as F
                text_embeds = F.normalize(outputs.pooler_output, p=2, dim=-1)
                return text_embeds
            else:
                # Fallback for base model
                return self.model.encode_text(input_ids, attention_mask)

    def encode_image(self, images):
        return self.model.encode_image(images)


def generate_visualizations(model, model_name, image_paths, image_embeds,
                          processor, transform, device, output_dir):
    """Generate all visualizations for a model."""

    print("\n" + "="*70)
    print(f"Generating visualizations for: {model_name}")
    print("="*70)

    model_output_dir = output_dir / model_name
    model_output_dir.mkdir(parents=True, exist_ok=True)

    # Wrap model if it's a modified model
    if isinstance(model, CLIPFineTuneModelModified):
        wrapped_model = ModifiedModelWrapper(model, processor)
    else:
        wrapped_model = model

    # Text→Image Retrieval
    print("\nREQUIREMENT 1: Text→Image Retrieval")
    print("-" * 70)

    text_queries = [
        "sport",
        "a dog playing",
        "a person eating",
        "a beautiful sunset",
        "a cat on a couch"
    ]

    for query in text_queries:
        print(f"  Query: '{query}'")
        save_path = model_output_dir / f"text2img_{query.replace(' ', '_')}.png"

        visualize_text_to_image_retrieval(
            query_text=query,
            model=wrapped_model,
            image_paths=image_paths,
            image_embeds=image_embeds,
            clip_processor=processor,
            device=device,
            top_k=5,
            save_path=save_path
        )

    # Grid view
    print(f"  Creating retrieval grid...")
    grid_path = model_output_dir / "text2img_grid.png"
    create_retrieval_grid(
        queries=text_queries[:4],
        model=wrapped_model,
        image_paths=image_paths,
        image_embeds=image_embeds,
        clip_processor=processor,
        device=device,
        images_per_query=5,
        save_path=grid_path
    )

    # Zero-Shot Classification
    print("\nREQUIREMENT 2: Zero-Shot Image Classification")
    print("-" * 70)

    class_labels = ['a person', 'an animal', 'a landscape']
    num_examples = 5
    sample_images = image_paths[:num_examples]

    for idx, img_path in enumerate(sample_images):
        print(f"  Classifying image {idx+1}/{num_examples}: {img_path.name}")
        save_path = model_output_dir / f"classification_example_{idx+1}.png"

        predicted_class, confidence = zero_shot_classification(
            query_image_path=img_path,
            class_labels=class_labels,
            model=wrapped_model,
            clip_processor=processor,
            transform=transform,
            device=device,
            save_path=save_path
        )

        print(f"    → Predicted: {predicted_class} ({confidence*100:.1f}%)")

    print(f"\n✓ All visualizations for {model_name} saved to: {model_output_dir}")


print("✓ All functions defined")

✓ All functions defined


## Load Model and Generate Visualizations

In [33]:
# Load CLIP processor
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
transform = get_clip_transform()

# Get validation image paths
all_image_paths = sorted(list(val_dir.glob("*.jpg")))[:num_images]
print(f"\n✓ Using {len(all_image_paths)} validation images")


✓ Using 1000 validation images


In [34]:
print("="*70)
print(f"Loading {MODEL_TYPE.upper()} Model")
print("="*70)

# Load model with correct architecture
model = load_model_with_architecture(model_checkpoint_path, MODEL_TYPE, device)

# Precompute embeddings
image_embeds = precompute_image_embeddings(model, all_image_paths, transform, device)

# Generate visualizations
generate_visualizations(
    model=model,
    model_name=MODEL_TYPE,
    image_paths=all_image_paths,
    image_embeds=image_embeds,
    processor=processor,
    transform=transform,
    device=device,
    output_dir=output_dir
)

# Clean up
del model, image_embeds
torch.cuda.empty_cache()

print(f"\n✅ {MODEL_TYPE} model visualizations complete!")

Loading BATCHNORM Model

Loading batchnorm model from /content/drive/MyDrive/elec475_lab4/models/best_model_batch_norm.pth...
⚠ Warning: Missing keys in checkpoint: ['text_encoder.embeddings.token_embedding.weight', 'text_encoder.embeddings.position_embedding.weight', 'text_encoder.encoder.layers.0.self_attn.k_proj.weight']...
⚠ Warning: Unexpected keys in checkpoint: ['text_encoder.text_model.embeddings.token_embedding.weight', 'text_encoder.text_model.embeddings.position_embedding.weight', 'text_encoder.text_model.encoder.layers.0.self_attn.k_proj.weight']...
✓ batchnorm model loaded and set to eval mode

Precomputing embeddings for 1000 images...
  Processed 320/1000
  Processed 640/1000
  Processed 960/1000
✓ Embeddings computed: torch.Size([1000, 512])

Generating visualizations for: batchnorm

REQUIREMENT 1: Text→Image Retrieval
----------------------------------------------------------------------
  Query: 'sport'
Saved visualization to /content/Visualizations/batchnorm/text2img

## Summary and Download Results

In [35]:
print("\n" + "="*70)
print("✅ VISUALIZATION GENERATION COMPLETE")
print("="*70)
print(f"\n📁 Output directory: {output_dir}")
print(f"\nGenerated visualizations:")

for subdir in output_dir.iterdir():
    if subdir.is_dir():
        num_files = len(list(subdir.glob("*.png")))
        print(f"\n  {subdir.name}/ ({num_files} images)")
        for file_path in sorted(subdir.glob("*.png"))[:5]:
            size_kb = file_path.stat().st_size / 1024
            print(f"    • {file_path.name} ({size_kb:.1f} KB)")
        if num_files > 5:
            print(f"    ... and {num_files - 5} more")

print("\n" + "="*70)
print("Lab 4 Section 2.4 Requirements:")
print("  ✓ Text→Image retrieval (including 'sport' and 'a dog playing')")
print("  ✓ Image classification with ['a person', 'an animal', 'a landscape']")
print("="*70)


✅ VISUALIZATION GENERATION COMPLETE

📁 Output directory: /content/Visualizations

Generated visualizations:

  batchnorm/ (11 images)
    • classification_example_1.png (920.0 KB)
    • classification_example_2.png (1309.3 KB)
    • classification_example_3.png (698.1 KB)
    • classification_example_4.png (765.3 KB)
    • classification_example_5.png (712.0 KB)
    ... and 6 more

Lab 4 Section 2.4 Requirements:
  ✓ Text→Image retrieval (including 'sport' and 'a dog playing')
  ✓ Image classification with ['a person', 'an animal', 'a landscape']


In [36]:
# Archive visualizations for download
import shutil

archive_name = f'/content/Lab4_Visualizations_{MODEL_TYPE}'
shutil.make_archive(archive_name, 'zip', output_dir)
print(f"\n✓ Created archive: {archive_name}.zip")

# Download the archive
files.download(f'{archive_name}.zip')
print("\n✓ Download started! Check your browser downloads.")


✓ Created archive: /content/Lab4_Visualizations_batchnorm.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓ Download started! Check your browser downloads.


## Optional: Copy to Google Drive

In [37]:
# Copy visualizations to Google Drive
import shutil

drive_output_dir = f'/content/drive/MyDrive/Lab4_Visualizations_{MODEL_TYPE}'

if os.path.exists(drive_output_dir):
    shutil.rmtree(drive_output_dir)

shutil.copytree(output_dir, drive_output_dir)
print(f"✓ Visualizations copied to Google Drive: {drive_output_dir}")

✓ Visualizations copied to Google Drive: /content/drive/MyDrive/Lab4_Visualizations_batchnorm
